---
Title: Customize Virtual Device Using Qemu.
Date: 2025-03-10 12:00
Lang: zh-cn
Category: Qemu
---

# 笔记介绍
该博客用于记录学习在qemu中添加自定义开发板的过程

**注意** qemu 官方在 `hw/misc/edu.c` 中提供了一个教学用的设备样例, 可参考此文件实现模拟设备创建.

## QEMU Object Model

在 qemu 中使用一套叫做 `Qemu Object Model` 的东西来实现面向对象式开发模拟设备，主要由这四个组件构成：
- Type：用来定义一个「类」的基本属性，例如类的名字、大小、构造函数等
- Class：用来定义一个「类」的静态内容，例如类中存储的静态数据、方法函数指针等
- Object：动态分配的一个「类」的具体的实例（instance），储存类的动态数据
- Property：动态对象数据的访问器（accessor），可以通过监视器接口进行检查

> 一个定义设备obj的示例
```c
static void pci_edu_register_types(void)
{
    static InterfaceInfo interfaces[] = {
        { INTERFACE_CONVENTIONAL_PCI_DEVICE },
        { },
    };
    static const TypeInfo edu_info = {
        .name          = TYPE_PCI_EDU_DEVICE,
        .parent        = TYPE_PCI_DEVICE,
        .instance_size = sizeof(EduState),
        .instance_init = edu_instance_init,
        .class_init    = edu_class_init,
        .interfaces = interfaces,
    };
    type_register_static(&edu_info);
}
type_init(pci_edu_register_types)
```

## QEMU添加设备流程

1. 在 `hw/{target_architecture}/` 下, 创建新设备定义文件
2. 实例化虚拟设备的 `TypeInfo`, 并提供 `type_register_static` 注册, `type_init` 调用
3. TODO

## QEMU API介绍

- `type_init` 宏, 会被扩展为`gcc attribute constructor`修饰的函数`do_qemu_init_##`函数, 此属性修饰的函数会在程序的`main()`函数之前自动调用, 其最终的作用是将一个 `TypeInfo` 结构体实例通过  `type_register_static` 函数注册到 `GHashTable` 中
